<a href="https://colab.research.google.com/github/umitkacar/genai-tutorials/blob/main/notebooks/gemma-3-1b-workout-summaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create workout summaries with Gemma 3 1B

<a target="_blank" href="https://colab.research.google.com/github/patrickloeber/genai-tutorials/blob/main/notebooks/gemma-3-1b-workout-summaries.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gemma 3 1B can be used for data captioning and summary creation tasks.

For example, personal push notifications in (mobile) apps are a good use case for this.

This notebook shows how to do it. We'll use an example workout data in json format and turn it into a fun workout summary.

Install the release for Gemma 3 and login to access the model.

In [1]:
!pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

  Cloning https://github.com/huggingface/transformers (to revision v4.49.0-Gemma-3) to /tmp/pip-req-build-s8m8ory1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-s8m8ory1
  Running command git checkout -q 367bab469b0ef32017e2a0a0a5dbac5d36002f03
  Resolved https://github.com/huggingface/transformers to commit 367bab469b0ef32017e2a0a0a5dbac5d36002f03
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10936468 sha256=6d068be9b671e80720fcd9370411de0d5620cd8f9fa6c8ede2a74526ab9ef9ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-734sc0p8/wheels/53/15/d5/d63b866c641d8863f9cd29a4cc7a5efc38476c3aae8247c195
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.3
    Uninstalling transf

In [17]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

In [18]:
!git config --global credential.helper store

Let's load the model and pipeline:

In [19]:
import torch
from transformers import AutoTokenizer, pipeline

model_id="google/gemma-3-1b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)

pipe = pipeline("text-generation", model="google/gemma-3-1b-it",
                device="cuda", torch_dtype=torch.bfloat16, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Device set to use cuda


Create the summary prompt and function to create the summary:

In [27]:
summary_prompt = """Bana bir hikaye yaz spor le ilgili uzun olsun paragraflı.
"""

def generate_workout_summary(workout_data):
  message = [
      {
          "role": "user",
          "content": [{"type": "text", "text": summary_prompt.format(data=workout_data)}]
      }
  ]
  output = pipe(message, max_new_tokens=50, do_sample=True)
  return output[0]['generated_text'][-1]["content"]

Let's feed in the workout data and try it:

In [28]:
workout_data = {
    "type": "run", "duration_minutes": 48, "avg_heart_rate_bpm": 150,
    "details": {"elevation_gain_m": 320, "pace_kmh": 10.2,
                "distance_km": 8.1, "personal_record": True}
}

summary = generate_workout_summary(workout_data)
print(summary)

Güneşin sıcaklığını hissetmemize rağmen, her şeyin her zaman gülebilmesi için iyi bir şey olmadığını bildiğim bir adam vardı: Sırfır. Sırfır, her şeyin bir anda yok ol


In [ ]:
from google.colab import drive
drive.mount('/content/drive')